In [1]:
# from sst and rainfall ---> rainfall
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt 
import tensorflow.compat.v1 as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv3D, Conv2D
from keras.layers import ConvLSTM2D
from keras.layers import Attention
from keras.layers import Input
from keras.models import Model
from keras.layers import Dropout, Lambda
from keras import optimizers
from keras.layers import Conv2DTranspose, TimeDistributed
from keras import callbacks
import xarray as xr

# Define global variables
lead_time = 5
save_path = "/home/cccr/roxy/matin/MTech_project/model/Conv-LSTM/7in6out/"
model_path = save_path + "ConvLstm_sst0rf_rf_5SI.h5"
add_data = "/home/cccr/roxy/matin/MTech_project/data/"

def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("1998-01-01","2016-12-31"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out



channels = ["SIFilteredrfBOB_0lag.nc","SIFilteredSSTBOB_0.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "SIFilteredrfBOB_0lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target

# Perform additional processing or modeling steps as needed
seq = tf.keras.Sequential()

# Add layers to the model
seq.add(ConvLSTM2D(filters=4, kernel_size=(3,3), padding='same', input_shape=(7,25,39,2), return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=False, data_format='channels_last'))
seq.add(Conv2D(filters=15, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
seq.add(Conv2D(filters=1, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
# Create the query, value, and key inputs
query_input = Input(shape=(25,39,2), name='query_input')
value_input = Input(shape=(25,39,2), name='value_input')
key_input = Input(shape=(25,39,2), name='key_input')

# Pass the inputs to the Attention layer
attention_output = Attention(name='attention')([query_input, value_input, key_input])
Adam = tf.keras.optimizers.Adam(learning_rate=10**-4)
seq.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mae'])

print(seq.summary())

# Define the number of training epochs
n_epochs = 200
logdir = save_path
# Define the TensorBoard callback for monitoring training progress
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=10, batch_size=10, write_graph=True)

# Define the ReduceLROnPlateau callback for reducing the learning rate when the model plateaus
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=10**-20)

# Define the ModelCheckpoint callback for saving the best model based on validation mean absolute error
checkpoint_callback_mae = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_mae',
                                                             mode='min', save_best_only=True)

# Define the ModelCheckpoint callback for saving the best model based on validation loss
checkpoint_callback_loss = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss',
                                                              mode='min', save_best_only=True)

# Define the EarlyStopping callback for stopping training when the model stops improving
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

# Define the TerminateOnNaN callback for terminating the training process if there is NaN value in any of the output
terminate_callback = tf.keras.callbacks.TerminateOnNaN()

# Train the model using the defined callbacks
history = seq.fit(x, y, epochs=n_epochs, validation_split=0.1,
                  batch_size=50, callbacks=[early_stop_callback,
                  checkpoint_callback_mae, checkpoint_callback_loss,
                  tb_callback, lr_callback, terminate_callback])
#Save the trained model
seq.save(model_path)

#Save the training history

np.save(f'{save_path}ConvLstm_sst0rf_rf_5SI.npy', history.history)               
              

Adding channel 0 with shape: (2306, 7, 1, 25, 39)
Adding channel 1 with shape: (2306, 7, 1, 25, 39)
INPUT SHAPE -->  (2306, 7, 2, 25, 39)
TARGET SHAPE -->  (2306, 1, 25, 39)


2023-03-20 15:32:26.422260: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 7, 25, 39, 4)      880       
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 7, 25, 39, 8)      3488      
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 7, 25, 39, 8)      4640      
                                                                 
 conv_lstm2d_3 (ConvLSTM2D)  (None, 7, 25, 39, 16)     13888     
                                                                 
 conv_lstm2d_4 (ConvLSTM2D)  (None, 25, 39, 16)        18496     
                                                                 
 conv2d (Conv2D)             (None, 25, 39, 15)        2175      
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 39, 1)         1

Epoch 45/200
42/42 [==============================] - 277s 7s/step - loss: 200.1974 - mae: 8.2379 - val_loss: 116.0359 - val_mae: 7.0437 - lr: 1.0000e-04
Epoch 46/200
42/42 [==============================] - 247s 6s/step - loss: 200.0327 - mae: 8.2318 - val_loss: 115.9325 - val_mae: 7.0550 - lr: 1.0000e-04
Epoch 47/200
42/42 [==============================] - 269s 6s/step - loss: 199.8752 - mae: 8.2230 - val_loss: 115.7622 - val_mae: 7.0453 - lr: 1.0000e-04
Epoch 48/200
42/42 [==============================] - 262s 6s/step - loss: 199.7184 - mae: 8.2019 - val_loss: 115.6709 - val_mae: 7.0598 - lr: 1.0000e-04
Epoch 49/200
42/42 [==============================] - 260s 6s/step - loss: 199.5494 - mae: 8.2022 - val_loss: 115.3356 - val_mae: 6.9988 - lr: 1.0000e-04
Epoch 50/200
42/42 [==============================] - 283s 7s/step - loss: 199.3947 - mae: 8.1977 - val_loss: 115.3211 - val_mae: 7.0408 - lr: 1.0000e-04
Epoch 51/200
42/42 [==============================] - 265s 6s/step - loss: 1

42/42 [==============================] - 272s 7s/step - loss: 190.6991 - mae: 7.7376 - val_loss: 106.9751 - val_mae: 6.7182 - lr: 1.0000e-04
Epoch 99/200
42/42 [==============================] - 258s 6s/step - loss: 190.5674 - mae: 7.8629 - val_loss: 106.0121 - val_mae: 6.4802 - lr: 1.0000e-04
Epoch 100/200
42/42 [==============================] - 261s 6s/step - loss: 190.3483 - mae: 7.7566 - val_loss: 106.1532 - val_mae: 6.5878 - lr: 1.0000e-04
Epoch 101/200
42/42 [==============================] - 262s 6s/step - loss: 190.2470 - mae: 7.7883 - val_loss: 105.8061 - val_mae: 6.5208 - lr: 1.0000e-04
Epoch 102/200
42/42 [==============================] - 243s 6s/step - loss: 190.0448 - mae: 7.8042 - val_loss: 105.7469 - val_mae: 6.5466 - lr: 1.0000e-04
Epoch 103/200
42/42 [==============================] - 249s 6s/step - loss: 189.9129 - mae: 7.7347 - val_loss: 105.7688 - val_mae: 6.5877 - lr: 1.0000e-04
Epoch 104/200
42/42 [==============================] - 257s 6s/step - loss: 189.7256 

Epoch 151/200
42/42 [==============================] - 253s 6s/step - loss: 184.5485 - mae: 7.4573 - val_loss: 100.4708 - val_mae: 6.3188 - lr: 1.0000e-04
Epoch 152/200
42/42 [==============================] - 242s 6s/step - loss: 184.5455 - mae: 7.5305 - val_loss: 100.1210 - val_mae: 6.2475 - lr: 1.0000e-04
Epoch 153/200
42/42 [==============================] - 248s 6s/step - loss: 184.4728 - mae: 7.4900 - val_loss: 99.8818 - val_mae: 6.1913 - lr: 1.0000e-04
Epoch 154/200
42/42 [==============================] - 272s 6s/step - loss: 184.3142 - mae: 7.4988 - val_loss: 99.8011 - val_mae: 6.1829 - lr: 1.0000e-04
Epoch 155/200
42/42 [==============================] - 259s 6s/step - loss: 184.2826 - mae: 7.4477 - val_loss: 99.8249 - val_mae: 6.2095 - lr: 1.0000e-04
Epoch 156/200
42/42 [==============================] - 253s 6s/step - loss: 184.1564 - mae: 7.4925 - val_loss: 99.6311 - val_mae: 6.1610 - lr: 1.0000e-04
Epoch 157/200
42/42 [==============================] - 273s 6s/step - loss

In [2]:
def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("2017-01-01","2018-12-31"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out



channels = ["SIFilteredrfBOB_0lag.nc","SIFilteredSSTBOB_0.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "SIFilteredrfBOB_0lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target



Adding channel 0 with shape: (232, 7, 1, 25, 39)
Adding channel 1 with shape: (232, 7, 1, 25, 39)
INPUT SHAPE -->  (232, 7, 2, 25, 39)
TARGET SHAPE -->  (232, 1, 25, 39)


In [3]:
model = tf.keras.models.load_model(model_path)
pred = model.predict(x)

actual = y.flatten()
prediction = pred.flatten()
corr = np.corrcoef(actual,prediction)
corr

8/8 [==============================] - 2s 94ms/step


array([[1.        , 0.01871628],
       [0.01871628, 1.        ]])